Questions:
* When handling categorical features. Do I need to use pd.factorize? see  https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772. I'm just using get_dummies() directly.
* When using simple imputer, I lose the column names. It turns the pd.DataFrame into a np.ndarray. Is this the right process, or are there other ways to handle missing data, NaN, Inf, numbers too big for INT32, etc?
* Should I do one-hot encoding before or after merging the different datasets (ex, application_train & previous_application)?
* For previous_application after I run the simple imputer I don't have any NaN in the np ndarray, but when I turn it into a pandas DataFrame it has 11109336. I need to turn it into a pandas DataFrame to merge it with application_train. What can I do?

June 7 through June 17
Future steps - Must do:
* Using the other data files (Feature Engineering, Create flat dataset)
* Data visualization
* Normalizing data
* Handling skewed data
Report:
* Preliminary results

June 18 through July 1
Report:
* WEEK1: I should have a skeleton of all the areas of the report to share with mentor.
* WEEK2: I should have a first full version of the report to share with mentor and ready to submit
Future steps - Nice to do:
* CREATE A scikit-learn Pipeline
* MAKE AN ENSEMBLE METHOD OF MULTIPLE MODELS
* Measuring model/learner performance using other evaluation metric (LR+, f1, r2)?
* Hyperparameter tunening (different values until I find a max, use higher percentage of training & validation data)
* Review Handling missing or invalid data, other better methods?

DONE:
* Handling missing or invalid data
* Split train & validation set
* Hyperparameter tunening
* Measuring model/learner performance using evaluation metric (auc_roc_score)

Preliminary results:
- Categorical values turned into numerical features with one-hot encoding scheme
- Fill missing or wrong values
- RFR Default values, only application training data, score: 0.591
- RFR Best values with GridSearchCV, only application training data, score: 0.722 <- 0.62
- RFR Best values with GridSearchCV, ALL available data, score: ?
- ???? Best values with GridSearchCV, ALL available data, score: ?
- Normalizing Data
- Unskewing Data

Abbreviations:
RFR: Random Forest Regressor
????: Another regressor

# **Sections:**
[1. Import libraries & support functions](#1)  
[2. Dataset preparation](#2)  
[3. Exploratory Data Analysis (EDA)](#3)  
&nbsp; [3.1 Application Train](#3.1)  
[4. Feature Engineering](#4)  
&nbsp; [4.1 Handle Skewed Continuous Data](#4.1)  
&nbsp; [4.2 Normalize Continuous Data](#4.2)  
&nbsp; [4.3 Handle Categorical Variables](#4.3)  
&nbsp; [4.4 Handle Missing Data](#4.4)  
&nbsp; [4.5 Create a Flat Dataset](#4.5)  
&nbsp;&nbsp; [4.5.1 Previous Applications](#4.5)  
[5. Split Data into Training and Validation](#5)  
[6. Hyperparameter Tuning](#6)  
[7. Model Fitting & Prediction](#7)  

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSVfile I/O (e.g. pd.read_csv)
import os
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly import tools
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
from sklearn.ensemble import RandomForestRegressor
# Add evaluation metric to measure the model's performance
# Regression metrics available:
# http://scikit-learn.org/stable/modules/classes.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics
# http://scikit-learn.org/stable/modules/model_evaluation.html#receiver-operating-characteristic-roc
# http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html
# Cannot use sklearn.metrics.accuracy_score as it is a Classification metric
from sklearn.metrics import make_scorer, r2_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

In [2]:
# Support functions
'''
def bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1] 
    xx = cnt_srs.head(limit).values[::-1] 
    if rev:
        yy = cnt_srs.tail(limit).index[::-1] 
        xx = cnt_srs.tail(limit).values[::-1] 
    if xlb:
        trace = go.Bar(y=xlb, x=xx, orientation = 'h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx, orientation = 'h', marker=dict(color=color))
    if return_trace:
        return trace 
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

def gp(col, title):
    df1 = data_train[data_train["TARGET"] == 1]
    df0 = data_train[data_train["TARGET"] == 0]
    a1 = df1[col].value_counts()
    b1 = df0[col].value_counts()
    
    total = dict(data_train[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x)*100 / total[x0[i]] for i,x in enumerate(a1.values)]
    y1 = [float(x)*100 / total[x1[i]] for i,x in enumerate(b1.values)]

    trace1 = go.Bar(x=a1.index, y=y0, name='Target : 1', marker=dict(color="#96D38C"))
    trace2 = go.Bar(x=b1.index, y=y1, name='Target : 0', marker=dict(color="#FEBFB3"))
    return trace1, trace2 
'''

'\ndef bar_hor(df, col, title, color, w=None, h=None, lm=0, limit=100, return_trace=False, rev=False, xlb = False):\n    cnt_srs = df[col].value_counts()\n    yy = cnt_srs.head(limit).index[::-1] \n    xx = cnt_srs.head(limit).values[::-1] \n    if rev:\n        yy = cnt_srs.tail(limit).index[::-1] \n        xx = cnt_srs.tail(limit).values[::-1] \n    if xlb:\n        trace = go.Bar(y=xlb, x=xx, orientation = \'h\', marker=dict(color=color))\n    else:\n        trace = go.Bar(y=yy, x=xx, orientation = \'h\', marker=dict(color=color))\n    if return_trace:\n        return trace \n    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)\n    data = [trace]\n    fig = go.Figure(data=data, layout=layout)\n    iplot(fig)\n\ndef gp(col, title):\n    df1 = data_train[data_train["TARGET"] == 1]\n    df0 = data_train[data_train["TARGET"] == 0]\n    a1 = df1[col].value_counts()\n    b1 = df0[col].value_counts()\n    \n    total = dict(data_train[col].value_counts())\n    x0 = a1.inde

In [3]:
# List available data files
print(os.listdir("../input"))

# Data exploration

# Load the application training dataset application_train.csv
data_train = pd.read_csv("../input/application_train.csv")

# Load the application testing dataset application_test.csv
data_test = pd.read_csv("../input/application_test.csv")

'''
# Display the first 5 records of application_train.csv
display(data_train.head())

# Display the first 5 records of application_test.csv
display(data_test.head())
'''

# DataFrame statistics summary for selected columns
data_train[["AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE"]].describe()
#print(len(data_train.columns))

['POS_CASH_balance.csv', 'bureau_balance.csv', 'application_train.csv', 'previous_application.csv', 'installments_payments.csv', 'credit_card_balance.csv', 'sample_submission.csv', 'application_test.csv', 'bureau.csv']


,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE
count,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05
mean,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05
std,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05
min,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04
25%,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05
50%,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05
75%,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05
max,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06


In [4]:
# Total number of records
print("Total number of records in the train dataset: {:,}".format(len(data_train)))
# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET).
print("Total number of features in the train dataset: {}".format(data_train.shape[1] - 2))

# Total number of records
print("Total number of records in the test dataset: {:,}".format(len(data_test)))
# Total number of features. Excluding the load ids (SK_ID_CURR). There is NO target variable (TARGET) in the test dataset.
print("Total number of features in the test dataset: {}".format(data_test.shape[1] - 1))

Total number of records in the train dataset: 307,511
Total number of features in the train dataset: 120
Total number of records in the test dataset: 48,744
Total number of features in the test dataset: 120


In [5]:
# GRAPHS ARE NOT DISPLAYING
'''
# Target Variable Distribution 
bar_hor(data_train, "TARGET", "Distribution of Target Variable" , ["#44ff54", '#ff4444'], h=350, w=600, lm=200, xlb = ['Target : 1','Target : 0'])

tr0 = bar_hor(data_train, "CODE_GENDER", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)
tr1, tr2 = gp('CODE_GENDER', 'Distribution of Target with Applicant Gender')

fig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Gender Distribution" , "Gender, Target=1" ,"Gender, Target=0"])
fig.append_trace(tr0, 1, 1);
fig.append_trace(tr1, 1, 2);
fig.append_trace(tr2, 1, 3);
fig['layout'].update(height=350, showlegend=False, margin=dict(l=50));
iplot(fig);
'''

'\n# Target Variable Distribution \nbar_hor(data_train, "TARGET", "Distribution of Target Variable" , ["#44ff54", \'#ff4444\'], h=350, w=600, lm=200, xlb = [\'Target : 1\',\'Target : 0\'])\n\ntr0 = bar_hor(data_train, "CODE_GENDER", "Distribution of CODE_GENDER Variable" ,"#f975ae", w=700, lm=100, return_trace= True)\ntr1, tr2 = gp(\'CODE_GENDER\', \'Distribution of Target with Applicant Gender\')\n\nfig = tools.make_subplots(rows=1, cols=3, print_grid=False, subplot_titles = ["Gender Distribution" , "Gender, Target=1" ,"Gender, Target=0"])\nfig.append_trace(tr0, 1, 1);\nfig.append_trace(tr1, 1, 2);\nfig.append_trace(tr2, 1, 3);\nfig[\'layout\'].update(height=350, showlegend=False, margin=dict(l=50));\niplot(fig);\n'

In [6]:
# Transforming skewed continuous features

In [7]:
# Normalizing numerical features
from sklearn.preprocessing import MinMaxScaler

In [8]:
# Turn categorical variables into numerical features using the one-hot encoding scheme

# Total number of features. Excluding the load ids (SK_ID_CURR) and the target variable (TARGET)
print("Total number of features of train data, before one-hot encoding: {}".format(data_train.shape[1] - 2))
# Total number of features. Excluding the load ids (SK_ID_CURR). The test dataset has NO target label (TARGET)
print("Total number of features of test data, before one-hot encoding: {}".format(data_test.shape[1] - 1))

# One-hot encoding
data_train_encoded = pd.get_dummies(data_train)
data_test_encoded = pd.get_dummies(data_test)

# Total number of features. Excluding the loan ids (SK_ID_CURR) and the target variable (TARGET)
print("Total number of features of train data, after one-hot encoding: {}".format(data_train_encoded.shape[1] - 2))
# Total number of features. Excluding the loan ids (SK_ID_CURR). The test dataset has NO target label (TARGET)
print("Total number of features of test data, after one-hot encoding: {}".format(data_test_encoded.shape[1] - 1))

# New list of features
#print(list(data_train_encoded.columns))

# Determine what columns are missing
train_list = list(data_train_encoded.columns)
test_list = list(data_test_encoded.columns)
difference = [e for e in train_list if e not in test_list]
print(difference)

# Add those columns to the test set will all zeros
data_test_encoded_complete = data_test_encoded
for e in difference:
    if e != 'TARGET':
        data_test_encoded_complete[e] = 0

print("Total number of features of test data, after one-hot encoding: {}".format(data_test_encoded_complete.shape[1] - 1))

Total number of features of train data, before one-hot encoding: 120
Total number of features of test data, before one-hot encoding: 120
Total number of features of train data, after one-hot encoding: 244
Total number of features of test data, after one-hot encoding: 241
['TARGET', 'CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown']
Total number of features of test data, after one-hot encoding: 244


In [9]:
# Separate the target label from the train dataset. The column we are interested is 'TARGET'. Name it target_train.
target_train = data_train['TARGET']
print("Training target label has {:,} samples".format(target_train.shape[0]))

# Remove target label from the train dataset and rename to features_train.
features_train = data_train_encoded.drop(['TARGET'], axis=1)

# Test data has no taget label 'TARGET' in the dataset
features_test = data_test_encoded_complete

Training target label has 307,511 samples


In [10]:
# Handle missing values
# Source: https://www.kaggle.com/dansbecker/handling-missing-values
# http://scikit-learn.org/dev/modules/generated/sklearn.impute.SimpleImputer.html

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
features_train = my_imputer.fit_transform(features_train)
features_test = my_imputer.fit_transform(features_test)

# Consider turning features_train & features_test into a pandas DataFrame, instead of being a numpy ndarray
# prev_app_imputed = pd.DataFrame(my_imputer.fit_transform(prev_app_encoded), columns=prev_app_encoded.columns)

### <a id="4.5.1">4.5.1 Feature Engineering - Create a Flat Dataset - Previous Applications</a>

In [35]:
# Load the previous applications dataset
prev_app = pd.read_csv("../input/previous_application.csv")
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Check if therer is any NaN value in the dataset
prev_app.isnull().sum().sum()

11109336

In [37]:
# Total number of features. Excluding the current loan ids (SK_ID_CURR) and the previous loan ids (SK_ID_PREV)
print("Total number of features of previous applications, before one-hot encoding: {}".format(prev_app.shape[1] - 2))
print("Previous applications has {:,} samples".format(prev_app.shape[0]))

Total number of features of previous applications, before one-hot encoding: 35
Previous applications has 1,670,214 samples


In [38]:
# Handle Categorical data
prev_app_encoded = pd.get_dummies(prev_app)
prev_app_encoded.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_Business development,NAME_CASH_LOAN_PURPOSE_Buying a garage,NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_Buying a home,NAME_CASH_LOAN_PURPOSE_Buying a new car,...,CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Auto technology,NAME_SELLER_INDUSTRY_Clothing,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Construction,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_Furniture,NAME_SELLER_INDUSTRY_Industry,NAME_SELLER_INDUSTRY_Jewelry,NAME_SELLER_INDUSTRY_MLM partners,NAME_SELLER_INDUSTRY_Tourism,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,2030495,271877,1730.430,17145.0,17145.0,0.0,17145.0,15,1,0.0,0.182832,0.867336,-73,35,12.0,365243.0,-42.0,300.0,-42.0,-37.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2802425,108129,25188.615,607500.0,679671.0,NaN,607500.0,11,1,NaN,NaN,NaN,-164,-1,36.0,365243.0,-134.0,916.0,365243.0,365243.0,1.0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2523466,122040,15060.735,112500.0,136444.5,NaN,112500.0,11,1,NaN,NaN,NaN,-301,-1,12.0,365243.0,-271.0,59.0,365243.0,365243.0,1.0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,2819243,176158,47041.335,450000.0,470790.0,NaN,450000.0,7,1,NaN,NaN,NaN,-512,-1,12.0,365243.0,-482.0,-152.0,-182.0,-177.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1784265,202054,31924.395,337500.0,404055.0,NaN,337500.0,9,1,NaN,NaN,NaN,-781,-1,24.0,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
# Check if therer is any NaN value in the dataset
prev_app_encoded.isnull().sum().sum()

10288585

In [40]:
# Total number of features. Excluding the current loan ids (SK_ID_CURR) and the previous loan ids (SK_ID_PREV)
print("Total number of features of previous applications, after one-hot encoding: {}".format(prev_app_encoded.shape[1] - 2))

Total number of features of previous applications, after one-hot encoding: 162


In [51]:
# Handle missing data
prev_app_imputed_tmp = my_imputer.fit_transform(prev_app_encoded)
# Check if therer is any NaN value in the nparray
print(type(prev_app_imputed_tmp))
np.argwhere(np.isnan(prev_app_imputed_tmp))

<class 'numpy.ndarray'>


array([], shape=(0, 2), dtype=int64)

In [52]:
prev_app_imputed = pd.DataFrame(prev_app_imputed_tmp, columns=prev_app_encoded.columns)
prev_app_imputed.head()

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_Business development,NAME_CASH_LOAN_PURPOSE_Buying a garage,NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_Buying a home,NAME_CASH_LOAN_PURPOSE_Buying a new car,...,CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Auto technology,NAME_SELLER_INDUSTRY_Clothing,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Construction,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_Furniture,NAME_SELLER_INDUSTRY_Industry,NAME_SELLER_INDUSTRY_Jewelry,NAME_SELLER_INDUSTRY_MLM partners,NAME_SELLER_INDUSTRY_Tourism,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,2030495.0,271877.0,1730.430,17145.0,17145.0,0.000000,17145.0,15.0,1.0,0.000000,0.182832,0.867336,-73.0,35.0,12.0,365243.000000,-42.000000,300.000000,-42.000000,-37.000000,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2802425.0,108129.0,25188.615,607500.0,679671.0,6697.402139,607500.0,11.0,1.0,0.079637,0.188357,0.773503,-164.0,-1.0,36.0,365243.000000,-134.000000,916.000000,365243.000000,365243.000000,1.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2523466.0,122040.0,15060.735,112500.0,136444.5,6697.402139,112500.0,11.0,1.0,0.079637,0.188357,0.773503,-301.0,-1.0,12.0,365243.000000,-271.000000,59.000000,365243.000000,365243.000000,1.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2819243.0,176158.0,47041.335,450000.0,470790.0,6697.402139,450000.0,7.0,1.0,0.079637,0.188357,0.773503,-512.0,-1.0,12.0,365243.000000,-482.000000,-152.000000,-182.000000,-177.000000,1.00000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0

In [53]:
# Check if therer is any NaN value in the dataset
prev_app.isnull().sum().sum()

11109336

In [46]:
# Check if therer is any NaN value in the nparray
print(type(features_train))
np.argwhere(np.isnan(features_train))

<class 'numpy.ndarray'>


array([], shape=(0, 2), dtype=int64)

In [47]:
data = pd.DataFrame(features_train, columns=data_train_encoded.drop(['TARGET'], axis=1).columns)
data.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,...,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,-2120.0,12.061091,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083037,0.262949,0.139376,0.02470,0.036900,0.972200,0.619200,0.014300,0.000000,0.069000,0.083300,0.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,-291.0,12.061091,1.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.311267,0.622246,0.510853,0.09590,0.052900,0.985100,0.796000,0.060500,0.080000,0.034500,0.291700,0.333300,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,-2531.0,26.000000,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502130,0.555912,0.729567,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,-2437.0,12.061091,1.0,1.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502130,0.650442,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,-3458.0,12.061091,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,2.0,11.0,0.0,0.0,0.0,0.0,1.0,1.0,0.502130,0.322738,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [49]:
print(type(data))
data.isnull().sum().sum()

<class 'pandas.core.frame.DataFrame'>


0

In [17]:
data.shape

(307511, 245)

In [18]:
print("Total number of features of previous applications, before one-hot encoding: {}".format(prev_app_imputed.shape[1] - 2))
print("Previous applications has {:,} samples".format(prev_app_imputed.shape[0]))
## count the number of previous applications for a given ID
prev_apps_count = prev_app_imputed[['SK_ID_CURR', 'SK_ID_PREV']].groupby('SK_ID_CURR').count()
#print("Previous applications count: {}".format(prev_apps_count))
prev_app_imputed['SK_ID_PREV'] = prev_app_imputed['SK_ID_CURR'].map(prev_apps_count['SK_ID_PREV'])
prev_app_imputed.head()

Total number of features of previous applications, before one-hot encoding: 162
Previous applications has 1,670,214 samples


,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_Business development,NAME_CASH_LOAN_PURPOSE_Buying a garage,NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_Buying a home,NAME_CASH_LOAN_PURPOSE_Buying a new car,...,CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Auto technology,NAME_SELLER_INDUSTRY_Clothing,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Construction,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_Furniture,NAME_SELLER_INDUSTRY_Industry,NAME_SELLER_INDUSTRY_Jewelry,NAME_SELLER_INDUSTRY_MLM partners,NAME_SELLER_INDUSTRY_Tourism,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,3,271877.0,1730.430,17145.0,17145.0,0.000000,17145.0,15.0,1.0,0.000000,0.182832,0.867336,-73.0,35.0,12.0,365243.000000,-42.000000,300.000000,-42.000000,-37.000000,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,6,108129.0,25188.615,607500.0,679671.0,6697.402139,607500.0,11.0,1.0,0.079637,0.188357,0.773503,-164.0,-1.0,36.0,365243.000000,-134.000000,916.000000,365243.000000,365243.000000,1.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,122040.0,15060.735,112500.0,136444.5,6697.402139,112500.0,11.0,1.0,0.079637,0.188357,0.773503,-301.0,-1.0,12.0,365243.000000,-271.000000,59.000000,365243.000000,365243.000000,1.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,23,176158.0,47041.335,450000.0,470790.0,6697.402139,450000.0,7.0,1.0,0.079637,0.188357,0.773503,-512.0,-1.0,12.0,365243.000000,-482.000000,-152.000000,-182.000000,-177.000000,1.00000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.

In [19]:
prev_app_imputed[prev_app_imputed['SK_ID_CURR'] == 271877]

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_Business development,NAME_CASH_LOAN_PURPOSE_Buying a garage,NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_Buying a home,NAME_CASH_LOAN_PURPOSE_Buying a new car,...,CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Auto technology,NAME_SELLER_INDUSTRY_Clothing,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Construction,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_Furniture,NAME_SELLER_INDUSTRY_Industry,NAME_SELLER_INDUSTRY_Jewelry,NAME_SELLER_INDUSTRY_MLM partners,NAME_SELLER_INDUSTRY_Tourism,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,3,271877.0,1730.430,17145.0,17145.0,0.0,17145.0,15.0,1.0,0.000000,0.182832,0.867336,-73.0,35.0,12.0,365243.000000,-42.000000,300.000000,-42.000000,-37.000000,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
735737,3,271877.0,68258.655,1800000.0,1754721.0,180000.0,1800000.0,18.0,1.0,0.101325,0.188357,0.773503,-472.0,55.0,36.0,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838,0.33257,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1567518,3,271877.0,12417.390,108400.5,119848.5,0.0,108400.5,14.0,1.0,0.000000,0.188357,0.773503,-548.0,196.0,12.0,365243.000000,-512.000000,-182.000000,-392.000000,-387.000000,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [20]:
## Average values for all other features in previous applications
prev_apps_avg = prev_app_imputed.groupby('SK_ID_CURR').mean()
prev_apps_avg.columns = ['p_' + col for col in prev_apps_avg.columns]
data = data.merge(right=prev_apps_avg.reset_index(), how='left', on='SK_ID_CURR')

In [21]:
data.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,...,p_CHANNEL_TYPE_Car dealer,p_CHANNEL_TYPE_Channel of corporate sales,p_CHANNEL_TYPE_Contact center,p_CHANNEL_TYPE_Country-wide,p_CHANNEL_TYPE_Credit and cash offices,p_CHANNEL_TYPE_Regional / Local,p_CHANNEL_TYPE_Stone,p_NAME_SELLER_INDUSTRY_Auto technology,p_NAME_SELLER_INDUSTRY_Clothing,p_NAME_SELLER_INDUSTRY_Connectivity,p_NAME_SELLER_INDUSTRY_Construction,p_NAME_SELLER_INDUSTRY_Consumer electronics,p_NAME_SELLER_INDUSTRY_Furniture,p_NAME_SELLER_INDUSTRY_Industry,p_NAME_SELLER_INDUSTRY_Jewelry,p_NAME_SELLER_INDUSTRY_MLM partners,p_NAME_SELLER_INDUSTRY_Tourism,p_NAME_SELLER_INDUSTRY_XNA,p_NAME_YIELD_GROUP_XNA,p_NAME_YIELD_GROUP_high,p_NAME_YIELD_GROUP_low_action,p_NAME_YIELD_GROUP_low_normal,p_NAME_YIELD_GROUP_middle,p_PRODUCT_COMBINATION_Card Street,p_PRODUCT_COMBINATION_Card X-Sell,p_PRODUCT_COMBINATION_Cash,p_PRODUCT_COMBINATION_Cash Street: high,p_PRODUCT_COMBINATION_Cash Street: low,p_PRODUCT_COMBINATION_Cash Street: middle,p_PRODUCT_COMBINATION_Cash X-Sell: high,p_PRODUCT_COMBINATION_Cash X-Sell: low,p_PRODUCT_COMBINATION_Cash X-Sell: middle,p_PRODUCT_COMBINATION_POS household with interest,p_PRODUCT_COMBINATION_POS household without interest,p_PRODUCT_COMBINATION_POS industry with interest,p_PRODUCT_COMBINATION_POS industry without interest,p_PRODUCT_COMBINATION_POS mobile with interest,p_PRODUCT_COMBINATION_POS mobile without interest,p_PRODUCT_COMBINATION_POS other with interest,p_PRODUCT_COMBINATION_POS others without interest
0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,-2120.0,12.061091,1.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083037,0.262949,0.139376,0.02470,0.036900,0.972200,0.619200,0.014300,0.000000,0.069000,0.083300,0.125000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0
1,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,-291.0,12.061091,1.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.311267,0.622246,0.510853,0.09590,0.052900,0.985100,0.796000,0.060500,0.080000,0.034500,0.291700,0.333300,...,0.0,0.0,0.0,0.333333,0.333333,0.000000,0.333333,0.0,0.0,0.000000,0.000000,0.333333,0.333333,0.0,0.0,0.0,0.0,0.333333,0.000000,0.000000,0.0,0.333333,0.666667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0
2,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,-2531.0,26.000000,1.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502130,0.555912,0.729567,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,...,0.0,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0
3,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,-2437.0,12.061091,1.0,1.0,0.0,1.0,0.0,0.0,2.0,2.0,2.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.502130,0.650442,0.510853,0.11744,0.0884

In [22]:
data.shape

(307511, 408)

In [23]:
print(list(data.columns))

['SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MO

In [24]:
prev_app_imputed[prev_app_imputed['SK_ID_CURR'] == 271877]

,SK_ID_PREV,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Consumer loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_XNA,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,FLAG_LAST_APPL_PER_CONTRACT_N,FLAG_LAST_APPL_PER_CONTRACT_Y,NAME_CASH_LOAN_PURPOSE_Building a house or an annex,NAME_CASH_LOAN_PURPOSE_Business development,NAME_CASH_LOAN_PURPOSE_Buying a garage,NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,NAME_CASH_LOAN_PURPOSE_Buying a home,NAME_CASH_LOAN_PURPOSE_Buying a new car,...,CHANNEL_TYPE_Car dealer,CHANNEL_TYPE_Channel of corporate sales,CHANNEL_TYPE_Contact center,CHANNEL_TYPE_Country-wide,CHANNEL_TYPE_Credit and cash offices,CHANNEL_TYPE_Regional / Local,CHANNEL_TYPE_Stone,NAME_SELLER_INDUSTRY_Auto technology,NAME_SELLER_INDUSTRY_Clothing,NAME_SELLER_INDUSTRY_Connectivity,NAME_SELLER_INDUSTRY_Construction,NAME_SELLER_INDUSTRY_Consumer electronics,NAME_SELLER_INDUSTRY_Furniture,NAME_SELLER_INDUSTRY_Industry,NAME_SELLER_INDUSTRY_Jewelry,NAME_SELLER_INDUSTRY_MLM partners,NAME_SELLER_INDUSTRY_Tourism,NAME_SELLER_INDUSTRY_XNA,NAME_YIELD_GROUP_XNA,NAME_YIELD_GROUP_high,NAME_YIELD_GROUP_low_action,NAME_YIELD_GROUP_low_normal,NAME_YIELD_GROUP_middle,PRODUCT_COMBINATION_Card Street,PRODUCT_COMBINATION_Card X-Sell,PRODUCT_COMBINATION_Cash,PRODUCT_COMBINATION_Cash Street: high,PRODUCT_COMBINATION_Cash Street: low,PRODUCT_COMBINATION_Cash Street: middle,PRODUCT_COMBINATION_Cash X-Sell: high,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,3,271877.0,1730.430,17145.0,17145.0,0.0,17145.0,15.0,1.0,0.000000,0.182832,0.867336,-73.0,35.0,12.0,365243.000000,-42.000000,300.000000,-42.000000,-37.000000,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
735737,3,271877.0,68258.655,1800000.0,1754721.0,180000.0,1800000.0,18.0,1.0,0.101325,0.188357,0.773503,-472.0,55.0,36.0,342209.855039,13826.269337,33767.774054,76582.403064,81992.343838,0.33257,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1567518,3,271877.0,12417.390,108400.5,119848.5,0.0,108400.5,14.0,1.0,0.000000,0.188357,0.773503,-548.0,196.0,12.0,365243.000000,-512.000000,-182.000000,-392.000000,-387.000000,0.00000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#print('p_AMT_ANNUITY' in list(data.xcolumns))
data[data['SK_ID_CURR'] == 271877]['p_AMT_DOWN_PAYMENT']

148238    60000.0
Name: p_AMT_DOWN_PAYMENT, dtype: float64

In [26]:
# Shuffle and split the data
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

# Split the 'features' and 'target label' data into training and validating sets
# REPLACED 'features_train' with 'data', after merging previous_applications into application_train
X_train, X_val, y_train, y_val = train_test_split(data,
                                                  target_train,
                                                  test_size=0.2,
                                                  random_state=42)

print("Original Training set has {:,} samples.".format(data.shape[0]))
print("After split Training set has {:,} samples.".format(X_train.shape[0]))
print("After split Validating set has {:,} samples.".format(X_val.shape[0]))
print("Testing set has {:,} samples.".format(data_test.shape[0]))

Original Training set has 307,511 samples.
After split Training set has 246,008 samples.
After split Validating set has 61,503 samples.
Testing set has 48,744 samples.


In [27]:
# Run GridSearchCV or fully train an estimator
run_mode = 'grid_search' # 'grid_search' or 'train_estimator'

In [28]:
# Run GridSearchCV
if run_mode == 'grid_search':
    perc_samples = 0.001
    print("Preparing to run Hyperparameters tunning with GridSearchCV using {0:.2f}% of the training samples".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    #features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model)
    estimator = RandomForestRegressor(n_jobs=-1,
                                      random_state=42,
                                      verbose=0)

    # Determine which Parameters to tune
    '''
    Tested so far:
    parameters = {
        'n_estimators': [9, 10, 11, 12, 13, 14, 15],
        'criterion': ['mse', 'mae'],
        'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7],
        'max_features': [0.01, 0.1, 0.25, 0.45, 0.5, 0.55, 0.6, 0.75],
        'min_samples_split': [2, 3, 4, 5],
        'warm_start': [False, True]
    }
    '''
    parameters = {
        'n_estimators': [115, 125, 135],
        'min_samples_leaf': [62, 75, 87],
        'max_features': [0.18, 0.2, 0.23],
        'min_samples_split': [2, 3],
    }

    # Create a scorer to measure hyperparameters performance
    scorer = make_scorer(roc_auc_score)

    # Create GridSearchCV grid object
    grid_obj = GridSearchCV(estimator=estimator, 
                            param_grid=parameters, 
                            scoring=scorer)

    # Fit the GridSearchCV grid object with the reduced training dataset and find the best hyperparameters
    start = time()
    grid_fit = grid_obj.fit(features_train_small, target_train_small)
    end = time()
    grid_fit_time = (end - start) / 60 # Ellapsed time in minutes
    print("GridSearchCV estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Get the best estimator
    best_est = grid_obj.best_estimator_
    print("Best Estimator: \n{}\n".format(best_est))

    # Get the best score
    best_score = grid_obj.best_score_
    print("Best Estimator Score: {}\n".format(best_score))

    # Get the best parameters
    best_params = grid_obj.best_params_
    print("Best Hyperparameters that yield the best score: \n{}\n".format(best_params))

    # Make predictions with unoptimized estimator on the validation set
    pred_val = (estimator.fit(features_train_small, target_train_small)).predict(features_val_small)
    print("Unoptimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))

    # Predict with the best estimator on the validation set
    best_pred_val = best_est.predict(features_val_small)
    print("Optimized Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, best_pred_val)))

    # Predict with the best estimator on the testing set
    #pred_test = best_est.predict(features_test)

Preparing to run Hyperparameters tunning with GridSearchCV using 0.10% of the training samples


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:501: FutureWarning:

From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.



ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# Train estimator
# TODO: rename or remove '_small', it might be '_full' or nothing
if run_mode == 'train_estimator':
    # Use the full training and validation datasets to fit the estimator with the best hyperparameters
    perc_samples = 1
    print("Preparing to train an estimator using {0:.2f}% of the training dataset".format(perc_samples * 100))
    features_train_small = X_train[:int(perc_samples * X_train.shape[0])]
    target_train_small = y_train[:int(perc_samples * y_train.shape[0])]
    features_val_small = X_val[:int(perc_samples * X_val.shape[0])]
    target_val_small = y_val[:int(perc_samples * y_val.shape[0])]
    features_test_small = features_test[:int(perc_samples * features_test.shape[0])]

    # Initialize the Estimator (Learner or Regression Model) with the best hyperparameters
    '''
    estimator = RandomForestRegressor(criterion='mae', # default='mse', VERY SLOW
                                      min_samples_split=2, # default=2
                                      warm_start=False) # default=False
    Best Performance:
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      ROC_AUC_SCORE:  0.7417496532130599 (% samples: 0.15)
                                      LEADERBOARD SCORE: 0.722 (NO improvement)

    estimator = RandomForestRegressor(n_estimators=100, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=3, # default=2
                                      min_samples_leaf=50, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      ROC_AUC_SCORE:  0.7286786065442892 (% samples: 0.1)
                                      LEADERBOARD SCORE: 0.722

    estimator = RandomForestRegressor(n_estimators=12, # default=10
                                      max_features=0.45, # default='auto'
                                      min_samples_leaf=2, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=2) # default=0
                                      LEADERBOARD SCORE: 0.62
    '''
    estimator = RandomForestRegressor(n_estimators=125, # default=10
                                      max_features=0.2, # default='auto'
                                      min_samples_split=2, # default=2
                                      min_samples_leaf=75, # default=1
                                      n_jobs=-1, # default=1
                                      random_state=42, # default=None
                                      verbose=0) # default=0

    # Fit the estimator with the training dataset
    start = time()
    estimator.fit(features_train_small, target_train_small)
    end = time()
    print("Estimator fit time: {0:.2f} minutes".format((end - start) / 60))

    # Predict with the validation dataset
    pred_val = estimator.predict(features_val_small)
    print("Estimator prediction score on Validation set: \t{}".format(roc_auc_score(target_val_small, pred_val)))
    
    # Predict using the 'test' dataset for submission
    pred_test = estimator.predict(features_test_small)
    #pred_test = estimator.predict(features_test)

    # Prepare prediction for submission
    submission = pd.DataFrame()
    # Need to replace data_test_encoded_complete
    submission['SK_ID_CURR'] = data_test_encoded_complete['SK_ID_CURR'][:int(perc_samples * features_test.shape[0])]
    submission['TARGET'] = pred_test
    submission.head()
    submission.to_csv('RFR.csv', index=False)